## Apriori Exercise

In [ ]:
# import pandas
import pandas as pd  

# from apyori import apriori
from apyori import apriori

In [ ]:
# load data 
movie_data = pd.read_csv('https://raw.githubusercontent.com/pirandello/apriori/master/movie_dataset.csv', 
                         header = None)

**Task:** Explore the dataset.

**Task:** Transform the dataframe to a list of lists (a suitable format for apyori).

**Task:** Instantiate apriori and set the input parameters based on following constraints:

    * only movies that have been purchased at least 40 times
    * the minimum confidence for the rules is 20%
    * the minumum lift is 3

**Task:** How many association rules did we obtained?

**Task:** Print the first association rule.

**Task:** Convert the association rules to a DataFrame.

> Use these columns: 
>    - title_1
>    - title_2
>    - support 
>    - confidence
>    - lift